# **ALERTA** !!
Esse notebook requer runtime de **GPU** !
Vá na aba superior direita do colab e mude o tipo do runtime

In [1]:
### Importações projeto
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from transformers import AutoTokenizer, AutoModel
import torch
import time

# Importações collab
from google.colab import drive
import os

In [2]:
base_path = '/content/drive'
drive.mount(base_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install contractions

In [4]:
# Caminho para salvar as chaves no Google Drive
ssh_drive_path = '/content/drive/My Drive/ssh_keys'

# Criar a pasta no Google Drive se não existir
if not os.path.exists(ssh_drive_path):
    os.makedirs(ssh_drive_path)

# Caminhos locais para as chaves no Colab
ssh_local_path = os.path.expanduser('~/.ssh')
if not os.path.exists(ssh_local_path):
    os.makedirs(ssh_local_path)

ssh_private_key = f'{ssh_drive_path}/id_rsa'
ssh_public_key = f'{ssh_drive_path}/id_rsa.pub'

# Verificar se as chaves já existem no Google Drive
if os.path.exists(ssh_private_key) and os.path.exists(ssh_public_key):
    # Copiar as chaves do Google Drive para o Colab
    !cp "{ssh_private_key}" ~/.ssh/id_rsa
    !cp "{ssh_public_key}" ~/.ssh/id_rsa.pub

    # Configurar permissões das chaves
    !chmod 600 ~/.ssh/id_rsa

    # Iniciar o agente SSH e adicionar a chave
    !eval "$(ssh-agent -s)"
    !ssh-add ~/.ssh/id_rsa

    # Testar a conexão com o GitHub
    print("Testando conexão com o GitHub...")
    !ssh -o StrictHostKeyChecking=no -T git@github.com

else:
    print("As chaves SSH não foram encontradas no Google Drive.\n")
    print("Por favor, execute os seguintes comandos UMA VEZ para configurar as chaves:")
    print("""
# Gerar as chaves SSH e salvar no Google Drive
!ssh-keygen -t rsa -b 4096 -C "seuemail@example.com" -f ~/.ssh/id_rsa -N ""
!cp ~/.ssh/id_rsa "{ssh_drive_path}/id_rsa"
!cp ~/.ssh/id_rsa.pub "{ssh_drive_path}/id_rsa.pub"

# Copie a chave pública e adicione no GitHub (Configurações > SSH and GPG Keys)
!cat ~/.ssh/id_rsa.pub

# Após isso, reinicie a célula para carregar as chaves

## AVISO ##
Se nao tiver ainda clonado o repositório, execute o seguinte também ao final:
os.chdir(base_path + '/' + 'MyDrive' + '/' + 'ProjetoML')
!git clone git@github.com:DiegoFleury/MI201-groupe-4.git

    """)

Agent pid 10000
Could not open a connection to your authentication agent.
Testando conexão com o GitHub...
Hi DiegoFleury! You've successfully authenticated, but GitHub does not provide shell access.


In [5]:
# Substituir (para o seu)
project_repo_path = 'MyDrive/ProjetoML'
repo_name = 'MI201-groupe-4'
full_repo_path = base_path + '/' + project_repo_path + '/' + repo_name

# Rode isso na primeira vez que for rodar (substituir pelo seu)
# os.chdir(full_repo_path)
# !git clone https://github.com/DiegoFleury/MI201-groupe-4.git

data_path = full_repo_path + '/' + 'data/setup'
project_path = full_repo_path + '/' + 'notebooks'

# Mudar o diretório para o projeto
os.chdir(project_path)

# Listar os arquivos na pasta para conferir
!ls

conclusion.ipynb     lexicRepr.ipynb  preProcessingAndEDA.ipynb  syntaticRepr.ipynb
groupe4projet.ipynb  models.ipynb     semanticRepr.ipynb


In [6]:
### Carregamento e Preprocessamento dos Dados
# Substituir pelo caminho real do dataset
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

In [7]:
train_data

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [8]:
test_data

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,All alone in this old house again. Thanks for...,positive
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive


In [9]:
# Exemplo de colunas esperadas: 'text', 'sentiment'
print("Exemplo de dados:")
print(train_data.head())

# Separar texto e rótulos
texts = train_data['text']
labels = train_data['sentiment']

# Mapear rótulos para valores numéricos
label_mapping = {'neutral': 0, 'positive': 1, 'negative': -1}
labels = labels.map(label_mapping)

Exemplo de dados:
       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  


In [10]:
# Dividir os dados em treino, validacao e teste
X_train, X_val, y_train, y_val = train_test_split(texts, labels, test_size=0.2, random_state=42)
X_test, y_test = test_data['text'], test_data['sentiment']
y_test = y_test.map(label_mapping)

In [11]:
# Por enquanto somente, pois senao da erro mais pra frente na conversao implicita de null pra float do pandas
def drop_nulls(X, y):
    """
    Remove valores nulos em X e y de forma consistente.

    Args:
        X (pd.Series ou np.ndarray): Features (entrada).
        y (pd.Series ou np.ndarray): Labels (saída).

    Returns:
        tuple: (X_clean, y_clean) sem valores nulos.
    """

    # Combine X e y em um DataFrame temporário
    data = pd.DataFrame({'X': X, 'y': y})

    # Remover linhas com valores nulos em qualquer das colunas
    data = data.dropna()

    # Separar novamente X e y após remover nulos
    X_clean = data['X']
    y_clean = data['y']

    return X_clean, y_clean

X_train, y_train = drop_nulls(X_train, y_train)
X_val, y_val = drop_nulls(X_val, y_val)
X_test, y_test = drop_nulls(X_test, y_test)

In [12]:
len(X_train), len(X_val), len(X_test)

(21983, 5497, 3534)

# **ALERTA**
Essa proxima parte é pesada ! Rode com a **GPU** !

Se você não fez isso ainda, o runtime vai desconectar...

In [13]:
# Teste com esse código se foi:
if torch.cuda.is_available():
    print("GPU disponível:", torch.cuda.get_device_name(0))
else:
    print("GPU não está ativada. Configure o runtime para usar GPU.")

GPU disponível: Tesla T4


## Pré-procesamento

In [14]:
import re

def remove_emoji(text):
    """
    Remove emojis do texto.
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags
        u'\U00002702-\U000027B0'  # other symbols
        u'\U000024C2-\U0001F251'  # enclosed characters
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [15]:
import contractions

def decontract(text):
    """
    Expande contrações como "don't" para "do not".
    """
    return contractions.fix(text)

In [16]:
def strip_all_entities(text):
    """
    Remove links, menções (@user), hashtags, e pontuação do texto.
    """
    # Remover links
    text = re.sub(r'http\S+', '', text)

    # Remover menções (@user)
    text = re.sub(r'@\S+', '', text)

    # Remover hashtags (#hashtag)
    text = re.sub(r'#\S+', '', text)

    # Remover quebras de linha e tabulações
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', ' ', text)
    text = re.sub(r'\t', ' ', text)

    # Remover pontuações
    text = re.sub(r'[^\w\s]', '', text)

    # Converter para minúsculas
    return text.lower()

In [17]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

# Certifique-se de baixar os recursos do nltk antes de usar:
nltk.download('wordnet')
nltk.download('omw-1.4')

def apply_lemmatization(text):
    """
    Aplica lematização para reduzir palavras à sua forma base.
    """
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Assume que o verbo é predominante
    return ' '.join(lemmatized_words)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [18]:
def preprocess_text_for_distilbert(examples):
    """
    Função para pré-processar texto para uso com DistilBERT.
    """
    text = examples if isinstance(examples, str) else examples['text']

    # 1. Remover emojis
    text = remove_emoji(text)  # Supondo que você já tenha essa função implementada

    # 2. Expandir contrações
    text = decontract(text)

    # 3. Remover entidades como links, mentions e hashtags
    text = strip_all_entities(text)

    # 4. Aplicar lematização
    text = apply_lemmatization(text)

    # 5. Garantir que o texto esteja limpo e pronto
    return text


In [19]:
### Geração de Embeddings com LLM
# Usando BERT como exemplo
model_name = 'distilbert-base-uncased' # 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Função para gerar embeddings
def generate_embeddings(texts):
    embeddings = []
    batch_size = 16 # Ajustar se precisar (spoiler: precisa !)
    MAX_LENGTH = 128

    total_texts = len(texts)
    last_percentage = 0
    start_time = time.time()

    for i in range(0, len(texts), batch_size):

      batch = texts[i:i + batch_size]
      preprocessed_batch = [preprocess_text_for_distilbert(text) for text in batch]

      inputs = tokenizer(preprocessed_batch, return_tensors='pt', truncation=True, padding=True, max_length=MAX_LENGTH).to(device)

      with torch.no_grad():
          outputs = model(**inputs)

          # Use a média ou CLS token para representar o batch
          batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
          embeddings.extend(batch_embeddings)

          # Calcular percentual concluído
          percentage = int((i + batch_size) / total_texts * 100)

          # Exibir progresso apenas quando atinge múltiplos de 10%
          if percentage >= last_percentage + 10 or i + batch_size >= total_texts:
              elapsed_time = time.time() - start_time
              last_percentage = percentage
              print(f"{percentage}% concluído. Tempo decorrido: {elapsed_time:.2f}s")

    return np.array(embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [20]:
# Gerar embeddings para treino e teste
print("Gerando embeddings para os dados de treino...")
X_train_embeddings = generate_embeddings(X_train)

Gerando embeddings para os dados de treino...
10% concluído. Tempo decorrido: 12.04s
20% concluído. Tempo decorrido: 14.37s
30% concluído. Tempo decorrido: 16.89s
40% concluído. Tempo decorrido: 19.43s
50% concluído. Tempo decorrido: 23.10s
60% concluído. Tempo decorrido: 26.08s
70% concluído. Tempo decorrido: 28.19s
80% concluído. Tempo decorrido: 30.28s
90% concluído. Tempo decorrido: 32.38s
100% concluído. Tempo decorrido: 34.48s


In [21]:
X_train_embeddings.shape

(21983, 768)

In [22]:
print("Gerando embeddings para os dados de validação...")
X_val_embeddings = generate_embeddings(X_val)

Gerando embeddings para os dados de validação...
10% concluído. Tempo decorrido: 0.53s
20% concluído. Tempo decorrido: 1.08s
30% concluído. Tempo decorrido: 1.63s
40% concluído. Tempo decorrido: 2.17s
50% concluído. Tempo decorrido: 2.75s
60% concluído. Tempo decorrido: 3.36s
70% concluído. Tempo decorrido: 3.92s
80% concluído. Tempo decorrido: 4.50s
90% concluído. Tempo decorrido: 5.10s
100% concluído. Tempo decorrido: 5.66s


In [23]:
print("Gerando embeddings para os dados de teste...")
X_test_embeddings = generate_embeddings(X_test)

Gerando embeddings para os dados de teste...
10% concluído. Tempo decorrido: 0.38s
20% concluído. Tempo decorrido: 0.71s
30% concluído. Tempo decorrido: 1.06s
40% concluído. Tempo decorrido: 1.41s
50% concluído. Tempo decorrido: 1.76s
60% concluído. Tempo decorrido: 2.10s
70% concluído. Tempo decorrido: 2.45s
80% concluído. Tempo decorrido: 2.79s
90% concluído. Tempo decorrido: 3.15s
100% concluído. Tempo decorrido: 3.49s


In [24]:
### Configuração e Teste de Modelos
# Dicionário de modelos para testar
models = {
    'SVM': SVC(),
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(max_iter=1000)
}

In [25]:
# Loop para treinar e avaliar cada modelo
results = {}
for model_name, model in models.items():
    print(f"Treinando o modelo: {model_name}")
    model.fit(X_train_embeddings, y_train)

    # Avaliação
    y_pred = model.predict(X_test_embeddings)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    results[model_name] = {'accuracy': accuracy, 'f1_score': f1}

    print(f"Resultados para {model_name}:")
    print(f"Acurácia: {accuracy:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


Treinando o modelo: SVM
Resultados para SVM:
Acurácia: 0.6740
F1-Score: 0.6744
              precision    recall  f1-score   support

          -1       0.68      0.65      0.66      1001
           0       0.63      0.72      0.67      1430
           1       0.75      0.64      0.69      1103

    accuracy                           0.67      3534
   macro avg       0.68      0.67      0.67      3534
weighted avg       0.68      0.67      0.67      3534

[[ 651  286   64]
 [ 230 1025  175]
 [  80  317  706]]
Treinando o modelo: Random Forest
Resultados para Random Forest:
Acurácia: 0.6019
F1-Score: 0.5998
              precision    recall  f1-score   support

          -1       0.62      0.51      0.56      1001
           0       0.56      0.71      0.62      1430
           1       0.68      0.54      0.61      1103

    accuracy                           0.60      3534
   macro avg       0.62      0.59      0.60      3534
weighted avg       0.61      0.60      0.60      3534

[[ 51

In [26]:
### Comparação de Resultados
results_df = pd.DataFrame(results).T
print("\nResumo dos Resultados:")
print(results_df)


Resumo dos Resultados:
                     accuracy  f1_score
SVM                  0.674024  0.674412
Random Forest        0.601868  0.599800
Logistic Regression  0.677136  0.677530


In [28]:
# Navegar para o repositório
os.chdir(full_repo_path)

!git config --global user.email "diegofleurycm@gmail.com"
!git config --global user.name "DiegoFleury"

# Verificar o status do repositório
!git status

# Adicionar mudanças
!git add .

# Fazer commit
!git commit -m "Atualização do projeto via Google Colab"

# Enviar para o GitHub
!git push


On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Everything up-to-date
